In [ ]:
import random
import sklearn
import numpy as np
import scipy
from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

N_POINTS = 10 # points in a small cluster
N_SMALL_CLUSTERS = 5 # small clusters in a big cluster
N_BIG_CLUSTERS = 5

In [ ]:
# Generating the clusters

def GenN(mean, cov):
    return scipy.stats.multivariate_normal(mean=mean, cov=cov).rvs()

def GenU(mean, dx):
    return [
        scipy.stats.uniform(loc=mean[0]-dx/2, scale=dx).rvs(),
        scipy.stats.uniform(loc=mean[1]-dx/2, scale=dx).rvs()
    ]

BIG_SIGMA = 5e1
SMALL_SIGMA = 1
POINTS_SIGMA = 5e-2

BIG_INTERCLUSTER_COV = np.array([[BIG_SIGMA**2, 0], [0, BIG_SIGMA**2]])
SMALL_INTERCLUSTER_COV = np.array([[SMALL_SIGMA**2, 0], [0, SMALL_SIGMA**2]])
POINTS_INTERCLUSTER_COV = np.array([[POINTS_SIGMA**2, 0], [0, POINTS_SIGMA**2]])

big_cluster_means = np.array([
    GenN([0, 0], BIG_INTERCLUSTER_COV)
    for _ in range(N_BIG_CLUSTERS)])

small_cluster_means = np.array([
    GenN(mean, SMALL_INTERCLUSTER_COV)
    for mean in big_cluster_means
    for _ in range(N_SMALL_CLUSTERS)
])

points = np.array([
    GenN(mean, POINTS_INTERCLUSTER_COV)
    for mean in small_cluster_means
    for _ in range(N_POINTS)
])

fig, ax = plt.subplots(1, 1, figsize=(8,6))
ax.plot(points[:,0], points[:,1], ".")
ax.plot(small_cluster_means[:,0], small_cluster_means[:,1], "r+")
ax.plot(big_cluster_means[:,0], big_cluster_means[:,1], "kx")
plt.show()

In [ ]:
# Aglomerative clustering

total_points = N_POINTS*N_SMALL_CLUSTERS*N_BIG_CLUSTERS
small_clusters = N_SMALL_CLUSTERS*N_BIG_CLUSTERS

clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=1, compute_distances=True, linkage="single")
clustering = clustering.fit(points)

In [ ]:
# Elbow

def AETParabolic(data, p, k):
    def f1(n):
        return n**2/2 + n/2

    def f2(n):
        return n**3/3 + n**2/2 + n/6

    def f4(n):
        return n**5/5 + n**4/2 + n**3/3 - n/30

    lsigma = f1(k)/k
    lv = f2(k) - k*lsigma**2
    fsigma = f2(k-1)/k
    fv = f4(k-1) - k*fsigma**2

    k0 = k-1

    while k0 < len(data):
        fsum = 0
        lsum = 0

        for i in range(k):
            fsum += (i**2 - fsigma) * (data[k0-k+1+i] + p*i)
            lsum += (i+1 - lsigma) * (data[k0-k+1+i] + p*i)

        delta = fsum**2/fv - lsum**2/lv

        if delta > 0:
            return k0
        else:
            k0 += 1
    return k0

In [ ]:
p_space = np.linspace(
    0, (clustering.distances_[-1] - clustering.distances_[0])/100, 1000)
k_values = [3, 4, 5, 7, 10, 20, 50, 100]
n_values = [None for _ in k_values]

for i in range(len(k_values)):
    n_values[i] = np.array(
        [total_points - AETParabolic(clustering.distances_, pv, k_values[i])
         for pv in p_space])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,5))

ax.plot([min(p_space), max(p_space)], [N_BIG_CLUSTERS, N_BIG_CLUSTERS], "r--")
ax.plot([min(p_space), max(p_space)], [small_clusters, small_clusters], "r--")

for i in range(len(k_values)):
    ax.plot(p_space, n_values[i], label=f"k={k_values[i]}")

ax.set_yscale("log")
ax.legend()

fig.subplots_adjust(left=0.08, right=0.98, top=0.98, bottom=0.05)
fig.savefig("./slides/figures/trend-analysis.eps")

In [ ]:
cluster_points = [
    total_points - small_clusters - 1,
    total_points - N_BIG_CLUSTERS - 1]

max_big_distance = max(clustering.distances_)
max_small_distance = max(clustering.distances_[:cluster_points[1]+1])

fig, ax = plt.subplots(figsize=(8, 4))

ax.plot(clustering.distances_, "k.")
ax.set_xlim([total_points - small_clusters-10, total_points+1])
ax.set_ylim([-max_big_distance*0.01, max_big_distance*1.05])
ax.plot(cluster_points, [clustering.distances_[i] for i in cluster_points], "r.")

fig.subplots_adjust(left=0.08, right=0.98, top=0.98, bottom=0.05)
fig.savefig("./slides/figures/agglomerative-elbow-1.eps")

fig, ax = plt.subplots(figsize=(8, 4))

ax.plot(clustering.distances_, "k.")
ax.set_xlim([total_points - small_clusters-10, total_points+1])
ax.set_ylim([-max_small_distance*0.05, max_small_distance*1.05])
ax.plot(cluster_points, [clustering.distances_[i] for i in cluster_points], "r.")

fig.subplots_adjust(left=0.08, right=0.98, top=0.98, bottom=0.05)
fig.savefig("./slides/figures/agglomerative-elbow-2.eps")